In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
from bs4 import BeautifulSoup
import urllib2
import requests

In [5]:
import json
with open("polinfo.json") as json_file:
    polinfo = json.load(json_file)
    json_file.close()

In [8]:
polinfo.keys()[0:5]

[u'candidate.php?candidate_id=4063',
 u'candidate.php?candidate_id=4062',
 u'candidate.php?candidate_id=4061',
 u'candidate.php?candidate_id=4792',
 u'candidate.php?candidate_id=4795']

In [367]:
#polinfo['candidate.php?candidate_id=4063']

In [368]:
soup=BeautifulSoup(polinfo['candidate.php?candidate_id=4063'],"html.parser")
#print soup.prettify()

In [25]:
rows = soup.find_all("a", attrs={"name": "movable_assets"})

In [227]:
soup=BeautifulSoup(polinfo['candidate.php?candidate_id=4063'],"html.parser")
rows = soup.find("a", attrs={"name": "movable_assets"}).find("table")

In [360]:
list_a = rows.find_all("tr", attrs={"valign": "top"})[4].find_all("td")
list_a

[<td><b>(b)</b><br>LIC or other insurance Policies <!--<br><span style='font-size:80%;color:yellow;background-color:red'>**Not counted in total assets</span> --></br></td>,
 <td align="right"><span class="desc">HDFC Life Insurance</span><br>  2,50,000\xa0<span style="color:brown;font-size:70%;text-decoration:none;white-space:nowrap">\xa02\xa0Lacs+</span> <br><br/></br></br></td>,
 <td align="right"><span class="desc">LIC Policy</span><br>  1,95,000\xa0<span style="color:brown;font-size:70%;text-decoration:none;white-space:nowrap">\xa01\xa0Lacs+</span> <br><br><span class="desc">HDFC Life Insurance Policy</span><br>  2,50,000\xa0<span style="color:brown;font-size:70%;text-decoration:none;white-space:nowrap">\xa02\xa0Lacs+</span> <br><br/></br></br></br></br></br></td>,
 <td align="right">Nil</td>,
 <td align="right">Nil</td>,
 <td align="right">Nil</td>,
 <td align="right"><span style="color:black"><b> Rs\xa06,95,000 </b></span><span style="color:brown;font-size:70%;text-decoration:none

In [390]:
def movable_assets(info,url):
    soup=BeautifulSoup(info[url],"html.parser")
    rows = soup.find("a", attrs={"name": "movable_assets"}).find("table")
   
    poldict={'srn':[],'description':[],'self':[],'spouse':[],'dependent1':[],'dependent2':[],'dependent3':[],'total':[]}
    
    row_count=len(rows.find_all("tr", attrs={"valign": "top"}))
    
    for i in range(0,row_count):
        list_a = rows.find_all("tr", attrs={"valign": "top"})[i].find_all("td")
        assets =[x.find_all(text=True,recursive=True) for x in list_a]
        if len(assets)>7:
            poldict['srn'].append(assets[0])
            poldict['description'].append(assets[1])
            poldict['self'].append(assets[2])
            poldict['spouse'].append(assets[3])
            poldict['dependent1'].append(assets[4])
            poldict['dependent2'].append(assets[5])
            poldict['dependent3'].append(assets[6])
            poldict['total'].append(assets[7])
        else:
            poldict['srn'].append("-")
            poldict['description'].append(assets[0])
            poldict['self'].append(assets[1])
            poldict['spouse'].append(assets[2])
            poldict['dependent1'].append(assets[3])
            poldict['dependent2'].append(assets[4])
            poldict['dependent3'].append(assets[5])
            poldict['total'].append(assets[6])
    
    return poldict
    

In [391]:
%%time
massets={}
for url in polinfo.keys():
    try:
        pdict=movable_assets(polinfo,url)
        massets[url]=pdict
    except TypeError: 
        print "error: ",url
        

error:  candidate.php?candidate_id=6112
error:  candidate.php?candidate_id=2923
error:  candidate.php?candidate_id=8068
error:  candidate.php?candidate_id=1469
error:  candidate.php?candidate_id=5058
error:  candidate.php?candidate_id=5390
error:  candidate.php?candidate_id=4001
CPU times: user 27min 56s, sys: 9min 33s, total: 37min 29s
Wall time: 57min 34s


In [392]:
#fd=open("movassets.json","w")
#json.dump(massets, fd)
#fd.close()

In [394]:
massets[polinfo.keys()[0]]

{'dependent1': [[u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u' Nil'],
  [u' Nil ']],
 'dependent2': [[u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u' Nil'],
  [u' Nil ']],
 'dependent3': [[u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u'Nil'],
  [u' Nil'],
  [u' Nil ']],
 'description': [[u'Cash'],
  [u'Deposits in Banks, Financial Institutions and Non-Banking Financial Companies'],
  [u'Bonds, Debentures and Shares in companies'],
  [u'(a)', u' NSS, Postal Savings etc'],
  [u'(b)',
   u'LIC or other insurance Policies ',
   u"<br><span style='font-size:80%;color:yellow;background-color:red'>**Not counted in total assets</span> "],
  [u'Personal loans/advance given '],
  [u'Motor Vehicles (details of make, etc.)'],
  [u'Jewellery (give details weight value)'],
  [u'Other assets, such as values of claims / inter

In [366]:
#example:
dft=pd.DataFrame(poldict)
dft.set_index('description',inplace=True)
dft

,dependent1,dependent2,dependent3,self,spouse,srn,total
description,,,,,,,
[Cash],[Nil],[Nil],[Nil],"[ 10,000 , 10 Thou+, ]","[ 10,000 , 10 Thou+, ]",[i],"[ Rs 20,000 , 20 Thou+]"
"[Deposits in Banks, Financial Institutions and Non-Banking Financial Companies]",[Nil],[Nil],[Nil],"[Current A/c , 3,50,000 , 3 Lacs+, ]","[Saving A/c , 75,000 , 75 Thou+, ]",[ii],"[ Rs 4,25,000 , 4 Lacs+]"
"[Bonds, Debentures and Shares in companies]",[Nil],[Nil],[Nil],[Nil],[Nil],[iii],[ Nil ]
"[(a), NSS, Postal Savings etc]",[Nil],[Nil],[Nil],[Nil],[Nil],[iv],[ Nil ]
"[(b), LIC or other insurance Policies , <br><span style='font-size:80%;color:yellow;background-color:red'>**Not counted in total assets</span> ]",[Nil],[Nil],[Nil],"[HDFC Life Insurance, 2,50,000 , 2 Lacs+, ]","[LIC Policy, 1,95,000 , 1 Lacs+, , HDFC Li...",totals,"[ Rs 6,95,000 , 6 Lacs+]"
[Personal loans/advance given ],[Nil],[Nil],[Nil],[Nil],[Nil],[v],[ Nil ]
"[Motor Vehicles (details of make, etc.)]",[Nil],[Nil],[Nil],"[Innova PY016X2866, 0*(Value Not Given) , ,...",[Nil],[vi],[ Rs 0 ]
[Jewellery (give details weight value)],[Nil],[Nil],[Nil],[Nil],"[100 Grams Gold, 17,00,000 , 17 Lacs+, ]",[vii],"[ Rs 17,00,000 , 17 Lacs+]"
"[Other assets, such as values of claims / interests]",[Nil],[Nil],[Nil],[Nil],[Nil],[viii],[ Nil ]


In [ ]:
"""
list_a = rows.find_all("tr", attrs={"valign": "top"})[0].find_all("td")
assets=[]
for a in list_a:
    if len(a.findAll("span"))==0:
        x=a.get_text().strip().encode('ascii', 'ignore')
        x=cleaner(x)
        assets.append(x)
    else:
        x=a.find_all("span")[0]
        y=x.previousSibling
        y=cleaner(y)
        assets.append(y)
assets"""